# Assignment 1 -- Julius Tabery, Harrison Le

## Task 1
#### Load the dataset and make at least two observations

In [1]:
import pandas as pd
import regex as re

In [2]:
raw_data = pd.read_csv('employer_raw_data_group_2.csv')

raw_data.describe()

,employers,description
count,20000,20000
unique,20000,19996
top,the outdoor circle ~,Computer Network Technology Corporation provid...
freq,1,2


In [3]:
print(raw_data['description'].loc[100])

“RehabAbilities has been designed with YOU in mind. Being therapist-owned, we speak your language & value your high clinical standards & ethics. We pride ourselves on having the most qualified & experienced Scheduling Team, Therapy Personnel, & Social Workers dedicated to providing excellent patient care! Physical Therapist Assistant Inpatient and Outpatient (Former Employee) - Corona, CA - March 24, 2021. RehabAbilities is a Pro white male racist company. After accepting assignments, assignments were often taken away from and given to white males.. Replacement assignments in lieu of the withdrawn assignments and with no additional compensation ... Find out what works well at RehabAbilities from the people who know best. Get the inside scoop on jobs, salaries, top office locations, and CEO insights. Compare pay for popular roles and read about the team’s work-life balance. Uncover why RehabAbilities is the best company for you. 1 review of RehabAbilities "After experiencing what I did 

### Observations
The dataset contains just two columns: the name of the company ("employers") and the description of the company ("description"). The "employers" column seems straightforward enough, but the "description" column is a bit more complicated. It all seems to pertain to the company, but the content of the descriptions seem to vary. For example, the description above starts with an advertisement for the company, but then also includes some kind of review from a former employee who accuses the company of racist and sexist practice. It seems like, often, pieces of information from distinct sources are separated by two or three periods ("..."). It seems like most numeric information will probably not be that helpful. Phone numbers, dates, mailing addresses, etc. I think, for the purposes of the model, these kinds of information will not be very useful.

## Task 2
#### Create the regex for a phone number

In [ ]:
'''
PATTERN BREAKDOWN:

(?:(?:\+1 ?)?|1 ?) 
Matches if there is "1" or "+1" at the beginning, since some phone numbers include this.
It also matches if there is a space after the "1" or "+1".

(?:\([0-9]{3}\)|[0-9]{3})
Matches a set of three numbers, possibly inside of parentheses.

[. \-]{0,3} 
Matches characters between the numbers, such as in examples 1, 3, 4, 5, and 6.

[0-9]{3} 
Matches 3 more numbers

[. \-]{0,3}
Matches more characters between the numbers

[0-9]{4} 
Matches 4 numbers at the end
'''

pattern = "(?:(?:\+1[ \-]?)?|1[ \-]?)(?:\([0-9]{3}\)|[0-9]{3})[. \-]{0,3}[0-9]{3}[. \-]{0,3}[0-9]{4}"

good_examples = []
good_examples.append("My phone number is +1 (123) 456 7890.")
good_examples.append("Here's my phone number: +11234567890.")
good_examples.append("You can reach me at +1(123)-456-7890.")
good_examples.append("My number is (123) - 456 - 7890.")
good_examples.append("Phone: 1 123- 456- 7890.")
good_examples.append("My phone: 123.456.7890.")
good_examples.append("Call my office: +1-(123)-456-7890.")
for example in good_examples:
    print("This should find a match.     Matches:", re.findall(pattern, example))

bad_examples = []
bad_examples.append("My phone number is +1 123) 456 7890.")   # Closing parenthesis without opening
bad_examples.append("Here's my phone number: 123456789.")     # Too short
bad_examples.append("You can reach me at +1 123-\n456-7890.") # Newline character in the middle
bad_examples.append("My number is (123) - 456 - 789.")        # Too short
bad_examples.append("Phone: 123*456*7890.")                   # Invalid character
bad_examples.append("My phone: 1 800-GET-RICH.")              # letters, not numbers
bad_examples.append("Call my office: +1-(123)-4567-890.")     # Numbers partitioned incorrectly
for example in bad_examples:
    print("This should NOT find a match. Matches:", re.findall(pattern, example))

This should find a match.     Matches: ['+1 (123) 456 7890']
This should find a match.     Matches: ['+11234567890']
This should find a match.     Matches: ['+1(123)-456-7890']
This should find a match.     Matches: ['(123) - 456 - 7890']
This should find a match.     Matches: ['1 123- 456- 7890']
This should find a match.     Matches: ['123.456.7890']
This should find a match.     Matches: ['+1-(123)-456-7890']
This should NOT find a match. Matches: []
This should NOT find a match. Matches: []
This should NOT find a match. Matches: []
This should NOT find a match. Matches: []
This should NOT find a match. Matches: []
This should NOT find a match. Matches: []
This should NOT find a match. Matches: []


## Task 3
#### Apply all the cleaning techniques on the dataset by using function. Your function will take a string as an input and will return the clean version of it. Create one function per regex + string manipulation you do. Use the apply function of pandas to clean your dataset.

In [ ]:
def clean_string(dirty_string):
    # This will return a cleaned version of dirty_string
    return

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=262b3d28-05ef-49db-b57a-efab2f090880' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>